## まずは健診データ（xls）をArcGISに読み込む ※xlsに事前に変換する必要あり
## 読み込んだらExcelToTable_conversionでgdb（ArcGISファイル形式）に変換

In [1]:
import os
import xlrd
import arcpy
def importallsheets(in_excel, out_gdb):
    workbook = xlrd.open_workbook(in_excel)
    sheets = [sheet.name for sheet in workbook.sheets()]
    print('{} sheets found: {}'.format(len(sheets), ','.join(sheets)))
    for sheet in sheets:
        # The out_table is based on the input excel file name
        # a underscore (_) separator followed by the sheet name
        out_table = os.path.join(
            out_gdb,
            arcpy.ValidateTableName(
                "{0}_{1}".format(os.path.basename(in_excel), sheet),
                out_gdb))
        print('Converting {} to {}'.format(sheet, out_table))
        # Perform the conversion
        arcpy.ExcelToTable_conversion(in_excel, out_table, sheet)
if __name__ == '__main__':
    importallsheets(r'J:\2015\cohre2015.xls',
                    r'C:\Users\地域包括ケア\Documents\ArcGIS\Projects\MyProject1\MyProject1.gdb')

1 sheets found: Sheet1
Converting Sheet1 to C:\Users\地域包括ケア\Documents\ArcGIS\Projects\MyProject1\MyProject1.gdb\cohre2015_xls_Sheet1


## ↑ 健診データがgdbファイル形式のテーブルとして保存された

# 郵便番号データを健診データと同じプロジェクトフォルダに移動（MyProject1.gdb）

In [11]:
import arcpy
import os
try:
    arcpy.env.workspace = r"I:\HDDFiles\GISData_Import\Shape形式\日本測地系\32" #郵便番号のポイント、ポリゴンデータ両方はいってるフォルダ

    # Set local variable
    outWorkspace = r"C:\Users\地域包括ケア\Documents\ArcGIS\Projects\MyProject1\MyProject1.gdb" #健診データが読み込まれたフォルダ
    # Use ListFeatureClasses to generate a list of shapefiles in the
    # workspace shown above.
    fcList = arcpy.ListFeatureClasses()
    # Execute CopyFeatures for each input shapefile
    for shapefile in fcList:
        outFeatureClass = os.path.join(outWorkspace, shapefile.strip(".shp")) #.shp以前のファイル名が名前になって保存される
        arcpy.CopyFeatures_management(shapefile, outFeatureClass)
except Exception as err:
    print(err.args[0])

## ↑郵便番号データが健診データと同じプロジェクトフォルダに移動された

# 読み込んだ健診データを郵便番号ポイントデータと結合
## おうなん、隠岐、雲南だけ残す（expression1)
## おうなんだけ残す（expression2)

In [47]:
import arcpy
try:
    # Set environment settings
    arcpy.env.workspace = r'C:\Users\地域包括ケア\Documents\ArcGIS\Projects\MyProject1\MyProject1.gdb' #健診データ(gdb形式)、郵便番号データ全部入ってるフォルダ
    arcpy.env.qualifiedFieldNames = False

    # Set local variables    
    inFeatures = "Zip_32_1" #郵便番号ポイントデータ
    layerName1 = "zip_All" #lyrファイルに変換後の名前
    layerName2 = "zip_Onan" #lyrファイルに変換後の名前
    joinField1 = "yu_bin" #郵便番号ポイントデータの郵便番号
    joinTable = "cohre2015_xls_Sheet1" #健診データ
    joinField2 = "v12" #健診データの郵便番号
    expression1 = "shi_code = '32449' or shi_code = '32528' or shi_code = '32209'" #おうなん、隠岐、雲南のみ
    expression2 = "shi_code = '32449'" #おうなんOnly
    outFeatures1 = "All2015Postal"
    outFeatures2 = "Onan2015Postal" #最終アウトプットファイル

    #Proはレイヤーファイル（lyr）に変換しないとこの後の
    #フィルター(SelectLayerByAttribute_management)ができないので変換
    arcpy.MakeFeatureLayer_management (inFeatures,  layerName1) 
    arcpy.MakeFeatureLayer_management (inFeatures,  layerName2) 
    #郵便番号ポイントデータ（zip_point)と健診データ（cohre2015_xls_Sheet1）を結合
    arcpy.AddJoin_management(layerName1, joinField1, joinTable, joinField2) 
    arcpy.AddJoin_management(layerName2, joinField1, joinTable, joinField2)
    
    #郵便番号が合わなかったデータは消す（健診対象外地区）Error
    arcpy.SelectLayerByAttribute_management(layerName1, "NEW_SELECTION", expression1) 
    arcpy.SelectLayerByAttribute_management(layerName2, "NEW_SELECTION", expression2)
    # Copy the layer to a new permanent feature class
    arcpy.CopyFeatures_management(layerName1, outFeatures1)
    arcpy.CopyFeatures_management(layerName2, outFeatures2)
except Exception as err:
    print(err.args[0])

## 結合できたデータをマップにドラッグ

# 健診データが郵便番号別にプロット
# All2015Postal（n=3504）とOnan2015Postal（n=970）

### 受診者数の密度解析 (エラー）

In [52]:
import arcpy
from arcpy import env
from arcpy.sa import *

# Set environment settings
env.workspace = r'C:\Users\地域包括ケア\Documents\ArcGIS\Projects\MyProject1\MyProject1'

# Set local variables
inFeatures = "Onan2015Postal"
#ここでは単純なポイント数（BMIなど選択するとそのパラメータの値に基づいた密度解析）
populationField = "NONE" 
cellSize = 60

# Create the Neighborhood Object
radius = 2500
myNbrCirc = NbrCircle(radius, "MAP")

# Execute PointDensity
outPdens = PointDensity(inFeatures, populationField, cellSize, 
                        myNbrCirc, "SQUARE_KILOMETERS")

# Save the output 
outPdens.save(r'C:\Users\地域包括ケア\Documents\ArcGIS\Projects\MyProject1\MyProject1')

RuntimeError: 